In [1]:

# Preparing Data
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error,mean_squared_log_error, roc_auc_score, accuracy_score, f1_score, precision_recall_curve, log_loss
# Load the rock mines dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data"
df = pd.read_csv(url)
column_names = ["sensor_" + str(i) for i in range(1, 61)] + ["target"]
df.columns = column_names
df = df.reset_index()
df = df.rename(columns={"index": "id"})
df['target'] = df['target'].map({'M': 1, 'R': 0})
x_data = df.iloc[:, :60]
x_data = x_data.drop('id', axis=1)
y_data = df['target']
##### Get Base Scoring
from sklearn.metrics import log_loss
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import*
kfold = StratifiedKFold(n_splits=10, random_state=None)
model = RandomForestClassifier()
scoring = 'accuracy'
results = cross_val_score(model, x_data, y_data, cv=kfold, scoring=scoring)
print(f'Scores: {results.mean():.2f} (+/- {results.std() * 2:.2f})')
from itertools import combinations
columns = x_data.columns
combs = []
## or with subsets with only 2-3 columns
for i in range(1, 3):
    combs += list(combinations(columns, i))
### Get Single Column Scoring
feature_score = []
feature_score_std = []
i = 1
for i, comb in enumerate(combs, start=1):
    print(f"Processing combination {i}/{len(combs)}", end="\r")
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    X = x_data[list(comb)].values
    y = y_data.values
    scores= []
    for train_idx, val_idx in kf.split(X, y):
        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]
        model = SVC(gamma="auto", probability=True, random_state=42)
        model.fit(X_train,  y_train)
        y_pred = model.predict(X_val)
        score = accuracy_score( y_val, y_pred)
        scores.append(score)
    score_avg = np.mean(scores)
    score_std = np.std(scores)
    feature_score.append(score_avg)
    feature_score_std.append(score_std)
df_scores = pd.DataFrame({'name': list(combs), 'score': feature_score, 'std': feature_score_std})
# Creating List of Top performing Features
sorted_df = df_scores.sort_values(by='score', ascending=False)
flattened_df = sorted_df['name'].explode()
top_feature_list = flattened_df.unique()[:20].tolist()
## Re-Evaluation
kfold = StratifiedKFold(n_splits=10, random_state=None)
model = RandomForestClassifier()
scoring = 'accuracy'
results = cross_val_score(model, x_data[top_feature_list], y_data, cv=kfold, scoring=scoring)
print(f'Scores: {results.mean():.2f} (+/- {results.std() * 2:.2f})')


Scores: 0.71 (+/- 0.27)
Scores: 0.67 (+/- 0.30)1770/1770
